In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from statistics import median
import pickle
import csv
import multiprocess
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("../")
from projects import project_list

In [2]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [3]:
folder_path = '../data/25_1_travis_data/'
write_path = 'datasets/'

In [6]:
for p in project_list:
    
    p_name = p.split('/')[1]
    data = pd.read_csv(folder_path + p)
    data = get_first_failures(data)
    sec = len(data)//10
    
    for i in range(9):
        sec_data = data.iloc[i*sec:i*sec+sec]
        b_ids = sec_data['tr_build_id'].tolist()
        print(p, i, len(b_ids))
        
    
    i = i+1
    sec_data = data.iloc[i*sec:]
    b_ids = sec_data['tr_build_id'].tolist()
    print(p, i, len(b_ids))
    print('\n\n')
        

BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 0 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 1 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 2 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 3 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 4 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 5 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 6 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 7 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 8 205
BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv 9 211



apache-tajo/apache-tajo.csv 0 208
apache-tajo/apache-tajo.csv 1 208
apache-tajo/apache-tajo.csv 2 208
apache-tajo/apache-tajo.csv 3 208
apache-tajo/apache-tajo.csv 4 208
apache-tajo/apache-tajo.csv 5 208
apache-tajo/apache-tajo.csv 6 208
apache-tajo/apache-tajo.csv 7 208
apache-tajo/apache-tajo.csv 8 208
apache-tajo/apache-tajo.csv 9 209



osmdroid-osmdroid/osmdroid-osmdroid.csv 0 213
osmdroid-osmdroid/osmdroid-osmdroid.csv 1 213
osmdroid-osmdroid/osmd

structr-structr/structr-structr.csv 0 388
structr-structr/structr-structr.csv 1 388
structr-structr/structr-structr.csv 2 388
structr-structr/structr-structr.csv 3 388
structr-structr/structr-structr.csv 4 388
structr-structr/structr-structr.csv 5 388
structr-structr/structr-structr.csv 6 388
structr-structr/structr-structr.csv 7 388
structr-structr/structr-structr.csv 8 388
structr-structr/structr-structr.csv 9 388



floragunncom-search-guard/floragunncom-search-guard.csv 0 397
floragunncom-search-guard/floragunncom-search-guard.csv 1 397
floragunncom-search-guard/floragunncom-search-guard.csv 2 397
floragunncom-search-guard/floragunncom-search-guard.csv 3 397
floragunncom-search-guard/floragunncom-search-guard.csv 4 397
floragunncom-search-guard/floragunncom-search-guard.csv 5 397
floragunncom-search-guard/floragunncom-search-guard.csv 6 397
floragunncom-search-guard/floragunncom-search-guard.csv 7 397
floragunncom-search-guard/floragunncom-search-guard.csv 8 397
floragunncom-search

apache-sling/apache-sling.csv 0 1504
apache-sling/apache-sling.csv 1 1504
apache-sling/apache-sling.csv 2 1504
apache-sling/apache-sling.csv 3 1504
apache-sling/apache-sling.csv 4 1504
apache-sling/apache-sling.csv 5 1504
apache-sling/apache-sling.csv 6 1504
apache-sling/apache-sling.csv 7 1504
apache-sling/apache-sling.csv 8 1504
apache-sling/apache-sling.csv 9 1505



SonarSource-sonar-java/SonarSource-sonar-java.csv 0 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 1 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 2 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 3 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 4 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 5 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 6 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 7 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 8 1627
SonarSource-sonar-java/SonarSource-sonar-java.csv 9 1635



killbill-killbill/killbill-killbill.csv 0 1812
killbil

In [5]:
data

,Unnamed: 0,tr_build_id,gh_project_name,gh_is_pr,gh_pr_created_at,gh_pull_req_num,gh_lang,git_merged_with,git_branch,gh_num_commits_in_push,...,tr_log_num_test_suites_run,tr_log_num_test_suites_ok,tr_log_num_test_suites_failed,tr_log_tests_failed,tr_log_testduration,tr_log_buildduration,tr_original_commit,tr_duration,tr_status,tr_jobs
0,937845,24379923,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,6.0.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,70329a4aaa53659e943e96cd5f2238f016489fbd,281.0,passed,[24379924]
1,937852,24380102,BuildCraft/BuildCraft,True,2014-05-04 09:06:50,1690.0,java,commits_in_master,6.0.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,daed4f98065baecc972a03707d2b26e71f7522ee,254.0,passed,[24380103]
2,937885,24380988,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,6.0.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ebde06a0cd90d0125835108ade3c713c7cabf203,242.0,passed,[24380989]
3,937991,24382814,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,6.0.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f4fa14b95685cada4a1d3942e1294c4c047bd319,244.0,passed,[24382815]
4,938012,24384624,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,6.0.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4def61f4506f1bef2bf9ca714fcb9209efb01f78,36.0,failed,[24384626]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,3174468,117145427,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,7.2.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f7f82648770ed6ed6f45c065cdf812599c75b3ce,180.0,passed,[117145428]
2052,3174499,117146209,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,7.2.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,a0a35561c71039fa22bb33783fdb525ff7e36e21,176.0,passed,[117146210]
2053,3174579,117149936,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,7.2.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,03ea0d8f260b4b9d755a2f96dce1b720ef01d955,169.0,passed,[117149937]
2054,3175937,117248379,BuildCraft/BuildCraft,False,NaN,NaN,java,NaN,8.0.x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,02132645e1ba8ea0d03e9c21b15d52bd3c354239,26.0,errored,[117248390]
